### Tutorial 5: Creating synthetic profiles

All tutorials so far have dealt with analysing real datasets, but `riserfit` also has some built-in functionalities for synthetic data analysis.
Virtually everything revolving around synthetic data is implemented in `riserfit.RiserPlayground`, a class inheriting its methods from `riserfit.Riser`. It has some additional methods to easily create large sets of synthetic profiles: `riserfit.RiserPlayground.create_profiles_from_parameters()` or to copy profiles directly from an existing `riserfit.Riser` instance: `riserfit.RiserPlayground.load_profiles_from_Riser()`. To test the effects of noise in elevation data, it is possible to add gaussian noise with a user-defined spatial resolution and magnitude to the profiles: `riserfit.RiserPlayground.add_gaussian_z_noise()`.

In [ ]:
# Some imports
import riserfit as rf
import os
import matplotlib.pyplot as plt
import numpy as np

In this notebook, we will test the influence of vertical noise on the robustness of diffusion age inversion. We restrict ourselves to linear diffusion to avoid high computation times.

We set up the test as follows: We choose three different diffusion ages: 10, 100, and 1000 m^2 and three different levels of noise: 0.05, 0.1, and 0.5 m. We create 100 profiles for each of the specifications and calculate kt. The results are then displayed in the form of histograms.

In [ ]:
# set up the figure
fg, ax = plt.subplots(3, 3, layout='constrained')
fg2, ax2 = plt.subplots(5, 1, layout='constrained')

np.random.seed(1)

# loop over diffusion ages:
for i, kt in enumerate([10, 100, 1000]):
    # loop over noise levels:
    for j, sigma in enumerate([0.25, 0.5, 0.75]):

        # Create a new riser playground instance
        pg = rf.RiserPlayground()

        # create profiles
        pg.create_profiles_from_parameters(
            d=np.arange(-200, 200, 1), # profile distance values, m
            kt=[kt]*100, # we want 100 profiles with true diffusion age kt
            a=5, # riser height, m
            b=0.0, # far-field slope, m/m
            theta=0.5, # initial slope, m/m
            use_linear=True # create these profiles assuming a linear diffusion model.
        )

        # add noise with specified amplitude
        pg.add_gaussian_z_noise(
            dx=10, # spacing of generated noise values, m
            std_z=sigma # std of Gaussian distribution, m
        )

        # perform the linear diffusion fit
        pg.compute_best_linear_diffusion_fit(
            b_range=(-0.05, 0.05), # range of allowed far-field slopes, m/m
            theta_range=(0.5, 0.5), # range of allowed initial slopes, m/m
            init_theta=0.5, # initial guess for initial slope (just to prevent a warning in this case)
            kt_range=(0, 2000), # it helps to check what a completely degraded riser looks like to determine the upper bound to kt!
            verbose=False # do not print results to console
        )

        # plot some of the profiles, best-fits, and "true" shapes
        if kt == 10 and sigma == 0.5:
            # plot 10 profiles and 5 best-fits...
            for k in range(0, 5):
                ax2[k].plot(pg.d[k], pg.z[k])
                ax2[k].plot(
                    pg.d[k], 
                    rf.analytical_profile(
                        pg.d[k], kt, 5, 0, 0.5
                    )
                )
                ax2[k].plot(
                    pg.d[k], 
                    rf.analytical_profile(
                        pg.d[k], 
                        pg.best_kt[k], 
                        pg.best_a[k], 
                        pg.best_b[k], 
                        pg.best_theta[k]
                    )
                )
                ax2[k].text(
                    0.01, 0.95, 
                    f"best-fit kt: {pg.best_kt[k]:.2f}, theta: {pg.best_theta[k]:.2f}", 
                    ha="left", va="top",
                    transform=ax2[k].transAxes
                )
                            
            
        # calculate mean result
        print(f"True kt: {kt}, sigma: {sigma}, median kt: {np.median(pg.best_kt):.2f}, std kt: {np.std(pg.best_kt):.2f}")
        
        # add histogram plot
        ax[i,j].hist(pg.best_kt, bins=15)
        ax[i,j].axvline(kt, ls="dashed", lw=1, c="black", label="True kt")
        ax[i,j].axvline(np.median(pg.best_kt), ls="dashed", lw=1, c="red", label="Median kt")
        ax[i,j].set_title(f"kt: {kt}, s: {sigma}", fontsize=8)
        ax[i,j].legend(frameon=False, loc="upper right", fontsize=8)
        #ax[i,j].set_xlim(0.5*kt, 1.5*kt)

fg.supxlabel("kt [m^2]", fontsize=10)
fg2.supxlabel("Distance [m]", fontsize=10)
fg2.supylabel("Elevation [m]", fontsize=10)
plt.show()

We see the expected patterns: Relative errors are larger for smaller kt, and errors generally are larger for higher sigma.

Experimenting with the above experiment can reveal some pitfalls of morphological dating. Here are some things you can try out to make the results significantly worse:

1. Increase the upper kt bound (`kt_range`) from 2000 m^2 to 10000 m^2 and observe how the standard deviation in calculated kt changes. Also look at the profiles that are the "best-fit" to the actual data. One of them is clearly way off. This is probably because the shape of a riser of high age (> 5000 m^2) only changes very slowly, leading to only minor improvements in misfit as kt changes. This causes our optimization algorithm to terminate early.

In [ ]:
# Altering the diffusion age by 100 barely changes the shape of the riser profile and thus almost does not change misfit.
# Our optimization algorithm thinks that we've found a minimum.

d = np.linspace(-200, 200, 100)
z1 = rf.analytical_profile(
    d, 8000, 5, 0., 0.5
)
z2 = rf.analytical_profile(
    d, 8100, 5, 0., 0.5
)

plt.plot(d, z1)
plt.plot(d, z2)
plt.show()

2. Change the initial slope constraint (`theta_range`) to (0.4, 0.6) and observe the results. This time, we systematically underestimate diffusion ages and initial slopes (although we have set the initial guess for theta to the correct value!). This is because there is no unique relationship between riser shape and initial slope for older profiles.

In [ ]:
d = np.linspace(-200, 200, 100)
z1 = rf.analytical_profile(
    d, 100, 5, 0., 0.5 # 100 m^2 old profile with an initial slope of 0.5 m/m
)
z2 = rf.analytical_profile(
    d, 70, 5, 0., 0.3 # 70 m^2 old profile with an initial slope of 0.3 m/m
)

plt.plot(d, z1)
plt.plot(d, z2)
plt.show()

To conclude, from this synthetic study we can get an understanding of how noise affects our results. We also learned about the importance of having good constraints on all our parameters before we attempt an inversion for diffusion age.